# Import related pacakages

In [2]:
import cobra
import sys
sys.path.append(r'./code/')
from cobrapy_ec_model_function import *

# Inputing files

In [1]:
# The genome-scale metabolic model for constructing the enzyme-constrained model
model_name = './data/iML1515.xml' 

# Reaction-kcat file.#数据转h-1,乘3600
# eg. AADDGT,49389.2889
#reaction_kcat_file = "./data/reaction_kcat.csv"
#reaction_kcat_file = "./data/reaction_kcat_smoment.csv"
reaction_kcat_file = "./data/reaction_kappori.csv"

# Gene-abundance file. 
# eg. b0789,1.1
gene_abundance_file = "./data/gene_abundance.csv"

# Gene-molecular_weight file. 
# eg. b0001,thrL,2.13846
gene_molecular_weight_file = "./data/gene_molecular_weight.csv"


# Step1: Preprocessing of model

The reversible reactions in the GEM model are divided into two irreversible reactions. The input is iML1515 with 2712 reactions. The output is a model with 3375 irreversible reactions.

In [3]:
model = cobra.io.read_sbml_model(model_name)
convert_to_irreversible(model)
model

Name,iML1515
Memory address,0x01e87c18def0
Number of metabolites,1877
Number of reactions,3375
Number of groups,38
Objective expression,1.0*BIOMASS_Ec_iML1515_core_75p37M - 1.0*BIOMASS_Ec_iML1515_core_75p37M_reverse_35685
Compartments,"cytosol, extracellular space, periplasm"


# Step2: Retrieving enzyme kinetics and proteomics data

The inputs are GEM model. The outputs are 'genes' and 'gpr_relationship' data in the iML1515.

In [4]:
[genes,gpr_relationship] = get_genes_and_gpr(model)

Get the molecular weight of the enzyme (MW) according to the file of all_reaction_GPR.csv, which obtained from the previous step (gpr_relationship, ./analysis/all_reaction_GPR.csv). We need to manually correct the error of the gene_reaction_rule of a small amount of reactions in iML1515 (See Supplementary Table S1 for details), and also need to manually get the subunit of each protein from EcoCyc.

In [5]:
# reaction-gene-subunit-MW file. 
# eg. ALATA_D2,D-alanine transaminase,b2551 or b0870,45.31659 or 36.49471 ,2 or 4 
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)

Calculate kcat/MW. The inputs are 'reaction_kcat' and 'reaction_MW' data for calculating the kcat/MW (When the reaction is catalyzed by several isozymes, the maximum is retained).

In [6]:
save_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = calculate_reaction_kcat_mw(reaction_kcat_file, reaction_mw, save_file)

Calculate f. The input is 'genes' data, 'gene_abundance.csv' and 'gene_molecular_weight.csv'.

In [7]:
f = calculate_f(genes, gene_abundance_file, gene_molecular_weight_file)
f

0.4059986079578236

# Step3: Save enzyme concentration constraint model as json file.

In [8]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma , lowerbound, upperbound)

In [59]:
json_model_path="./model/iML1515_irr_enz_constraint_app.json"
enz_model=get_enzyme_constraint_model(json_model_path)
fba_solution = enz_model.optimize()
fba_solution_df = fba_solution.to_frame()
fba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.4191429334139575

# Step4: Calibration parameters

In [18]:
%%time
# 10%酶量测试
import pandas as pd
import cobra

reaction_kcat_mw_file="./analysis/reaction_kcat_mw.csv"
reaction_kcat_mw = pd.read_csv(reaction_kcat_mw_file, index_col=0)
json_model_path="./model/iML1515_irr_enz_constraint_app.json"
norm_model=cobra.io.json.load_json_model(json_model_path)
norm_biomass=norm_model.slim_optimize()

df_biomass = pd.DataFrame()
df_biomass_select = pd.DataFrame()
for r in norm_model.reactions:
    with norm_model as model:
        if r.id in list(reaction_kcat_mw.index):
            r.bounds = (0, reaction_kcat_mw.loc[r.id,'kcat_MW']*0.0228)
            df_biomass.loc[r.id,'biomass'] = model.slim_optimize()
            biomass_diff = norm_biomass-model.slim_optimize()
            biomass_diff_ratio = (norm_biomass-model.slim_optimize())/norm_biomass
            df_biomass.loc[r.id,'biomass_diff'] = biomass_diff
            df_biomass.loc[r.id,'biomass_diff_ratio'] = biomass_diff_ratio
            if biomass_diff > 0.001:
                df_biomass_select.loc[r.id,'biomass_diff'] = biomass_diff
                df_biomass_select.loc[r.id,'biomass_diff_ratio'] = biomass_diff_ratio

df_biomass = df_biomass.sort_values(by="biomass_diff",axis = 0,ascending = False)
df_biomass.to_csv('./analysis/df_biomass.csv')

df_biomass_select = df_biomass_select.sort_values(by="biomass_diff",axis = 0,ascending = False)
df_biomass_select

Wall time: 12.1 s


,biomass_diff,biomass_diff_ratio
PDH,0.015870,0.018096
AKGDH,0.009544,0.010883
GAPD,0.001297,0.001479


In [19]:
kcat_data_colect_file="./data/kcat_data_colect.csv"
kcat_data_colect = pd.read_csv(kcat_data_colect_file, index_col=0)
kcat_data_colect.head(10)

,smoment_no_adj_kcat,smoment_adj_kcat,kcat_in_vitro,kapp,kcat_GO
TPI,12046.600000,12046.600000,61.647843,72.000000,61.647843
TPI_reverse,12046.600000,12046.600000,44.122226,36.794028,44.122226
HYD2pp,NaN,NaN,112.694878,2002.949016,112.694878
ATPS4rpp,107.387778,115.685193,20.000000,1876.341742,20000.000000
MDH,518.347059,489.969840,931.000000,7.200000,931.000000
PPTHpp,NaN,NaN,747.236867,1711.332852,747.236867
GGGABAH,NaN,NaN,27.764922,1615.094328,27.764922
PSP_L,166.807695,183.878381,1.430000,1481.000000,7.150000
HYD1pp,NaN,NaN,138.656986,1464.427652,138.656986
PGK,276.128238,272.921505,1480.000000,22.670191,1480.000000


In [23]:
need_change_reaction=list(df_biomass_select.index)
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)
reaction_kcat_file = "./data/reaction_kappori.csv"
reaction_kappori = pd.read_csv(reaction_kcat_file, index_col=0)

round_1_reaction_kapp_change = pd.DataFrame()
round_1_reaction_kcatmw_change = pd.DataFrame()
for eachreaction in need_change_reaction:
    kcat_ori = reaction_kappori.loc[eachreaction,'kcat']
    if kcat_ori < np.max(kcat_data_colect.loc[eachreaction,:]) * 3600:
        reaction_kappori.loc[eachreaction,'kcat'] = np.max(kcat_data_colect.loc[eachreaction,:]) * 3600
    reaction_kcat_mw = pd.DataFrame()
    mw = reaction_mw.loc[eachreaction, 'MW'].split('or')
    min_mw = min(map(float, mw))
    kcat_mw = reaction_kappori.loc[eachreaction, 'kcat'] / min_mw
    round_1_reaction_kcatmw_change.loc[eachreaction,'kcat'] = reaction_kappori.loc[eachreaction,'kcat']
    round_1_reaction_kcatmw_change.loc[eachreaction,'MW'] = min_mw
    round_1_reaction_kcatmw_change.loc[eachreaction,'kcat_MW'] = kcat_mw
    for r in norm_model.reactions:
        with norm_model as model:
            if r.id == eachreaction:
                r.bounds = (0, kcat_mw*0.0228)
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_ori'] = kcat_ori
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_change'] = reaction_kappori.loc[eachreaction,'kcat']
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_mw_new'] = kcat_mw
                round_1_reaction_kapp_change.loc[eachreaction,'norm_biomass'] = norm_biomass
                round_1_reaction_kapp_change.loc[eachreaction,'ori_biomass'] = df_biomass.loc[eachreaction,'biomass']
                round_1_reaction_kapp_change.loc[eachreaction,'new_biomass'] = model.slim_optimize()

save_file = "./analysis/reaction_kcatmw_change_round_1.csv"
round_1_reaction_kcatmw_change.to_csv(save_file)
save_file2 = "./analysis/reaction_kapp_change_round_1_cb.csv"
round_1_reaction_kapp_change.to_csv(save_file2)

In [87]:
need_change_reaction=list(df_biomass_select.index)
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)
reaction_kcat_file = "./data/reaction_kappori.csv"
reaction_kappori = pd.read_csv(reaction_kcat_file, index_col=0)

round_1_reaction_kapp_change = pd.DataFrame()
round_1_reaction_kcatmw_change = pd.DataFrame()
for eachreaction in need_change_reaction:
    kcat_ori = reaction_kappori.loc[eachreaction,'kcat']
    kcat_go = kcat_data_colect.loc[eachreaction,'kcat_GO'] * 3600
    if kcat_ori < kcat_go:
        reaction_kappori.loc[eachreaction,'kcat'] = kcat_go
    reaction_kcat_mw = pd.DataFrame()
    mw = reaction_mw.loc[eachreaction, 'MW'].split('or')
    min_mw = min(map(float, mw))
    kcat_mw = reaction_kappori.loc[eachreaction, 'kcat'] / min_mw
    round_1_reaction_kcatmw_change.loc[eachreaction,'kcat'] = reaction_kappori.loc[eachreaction,'kcat']
    round_1_reaction_kcatmw_change.loc[eachreaction,'MW'] = min_mw
    round_1_reaction_kcatmw_change.loc[eachreaction,'kcat_MW'] = kcat_mw
    for r in norm_model.reactions:
        with norm_model as model:
            if r.id == eachreaction:
                r.bounds = (0, kcat_mw*0.0228)
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_ori'] = kcat_ori
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_change'] = reaction_kappori.loc[eachreaction,'kcat']
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_mw_new'] = kcat_mw
                round_1_reaction_kapp_change.loc[eachreaction,'norm_biomass'] = norm_biomass
                round_1_reaction_kapp_change.loc[eachreaction,'ori_biomass'] = df_biomass.loc[eachreaction,'biomass']
                round_1_reaction_kapp_change.loc[eachreaction,'new_biomass'] = model.slim_optimize()

save_file = "./analysis/reaction_kcatmw_change_round_2.csv"
round_1_reaction_kcatmw_change.to_csv(save_file)
save_file2 = "./analysis/reaction_kapp_change_round_2_cb.csv"
round_1_reaction_kapp_change.to_csv(save_file2)

In [94]:
need_change_reaction=list(df_biomass_select.index)
reaction_gene_subunit_MW = "./data/reaction_gene_subunit_MW.csv"
reaction_mw = calculate_reaction_mw(reaction_gene_subunit_MW)
reaction_kcat_file = "./data/reaction_kappori.csv"
reaction_kappori = pd.read_csv(reaction_kcat_file, index_col=0)

round_1_reaction_kapp_change = pd.DataFrame()
round_1_reaction_kcatmw_change = pd.DataFrame()
for eachreaction in need_change_reaction:
    kcat_ori = reaction_kappori.loc[eachreaction,'kcat']
    kcat_go = kcat_data_colect.loc[eachreaction,'smoment_adj_kcat'] * 3600
    if kcat_ori < kcat_go:
        reaction_kappori.loc[eachreaction,'kcat'] = kcat_go
    reaction_kcat_mw = pd.DataFrame()
    mw = reaction_mw.loc[eachreaction, 'MW'].split('or')
    min_mw = min(map(float, mw))
    kcat_mw = reaction_kappori.loc[eachreaction, 'kcat'] / min_mw
    round_1_reaction_kcatmw_change.loc[eachreaction,'kcat'] = reaction_kappori.loc[eachreaction,'kcat']
    round_1_reaction_kcatmw_change.loc[eachreaction,'MW'] = min_mw
    round_1_reaction_kcatmw_change.loc[eachreaction,'kcat_MW'] = kcat_mw
    for r in norm_model.reactions:
        with norm_model as model:
            if r.id == eachreaction:
                r.bounds = (0, kcat_mw*0.0228)
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_ori'] = kcat_ori
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_change'] = reaction_kappori.loc[eachreaction,'kcat']
                round_1_reaction_kapp_change.loc[eachreaction,'kcat_mw_new'] = kcat_mw
                round_1_reaction_kapp_change.loc[eachreaction,'norm_biomass'] = norm_biomass
                round_1_reaction_kapp_change.loc[eachreaction,'ori_biomass'] = df_biomass.loc[eachreaction,'biomass']
                round_1_reaction_kapp_change.loc[eachreaction,'new_biomass'] = model.slim_optimize()

save_file = "./analysis/reaction_kcatmw_change_round_3.csv"
round_1_reaction_kcatmw_change.to_csv(save_file)
save_file2 = "./analysis/reaction_kapp_change_round_3_cb.csv"
round_1_reaction_kapp_change.to_csv(save_file2)

In [100]:
model_file = './data/iML1515.xml' 
reaction_kcat_mw_file="./analysis/reaction_kcatmw_change_round_2.csv"
#The enzyme mass fraction 
f = 0.406
# The total protein fraction in cell.
ptot = 0.56 
# The approximated average saturation of enzyme.
#sigma = 0.5 
sigma = 1#kcapp数据，饱和度为1
# Lowerbound  of enzyme concentration constraint. 
lowerbound = 0   
upperbound = round(ptot * f * sigma, 3)

trans_model2enz_json_model(model_file, reaction_kcat_mw_file, f, ptot, sigma , lowerbound, upperbound)

In [96]:
json_model_path="./model/iML1515_irr_enz_constraint.json"
enz_model=get_enzyme_constraint_model(json_model_path)
norm_model=cobra.io.json.load_json_model(json_model_path)
norm_model_fba_solution = norm_model.optimize()
enz_model_fba_solution = enz_model.optimize()
norm_model_fba_solution = norm_model_fba_solution.to_frame()
print(norm_model_fba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])
enz_model_fba_solution = enz_model_fba_solution.to_frame()
print(enz_model_fba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M'])


0.8769972144269698
0.8769972144269688


In [90]:
norm_model_fba_solution_select = norm_model_fba_solution[norm_model_fba_solution['fluxes']>0]
print(norm_model_fba_solution_select.shape)
enz_model_fba_solution_select = enz_model_fba_solution[enz_model_fba_solution['fluxes']>0]
print(enz_model_fba_solution_select.shape)

(439, 2)
(432, 2)


In [97]:
c13reaction_file = './data/C13reaction.csv' 
c13reaction = pd.read_csv(c13reaction_file, index_col=0)
c13reaction = list(c13reaction.index)

norm_model_fba_solution_select_id = list(norm_model_fba_solution_select.index)
enz_model_fba_solution_select_id = list(enz_model_fba_solution_select.index)

print (list(set(c13reaction).difference(set(norm_model_fba_solution_select_id))))
print (list(set(c13reaction).difference(set(enz_model_fba_solution_select_id))))

['MALS', 'PTAr', 'ICL', 'PFL', 'FBA', 'ACKr_reverse', 'PFK']
['MALS', 'PYK', 'ICL', 'PFL', 'ACKr_reverse', 'PTAr']


In [98]:
enz_model_fba_solution.fluxes['AKGDH']

5.970062721906744

In [99]:
enz_model_fba_solution.fluxes['PDH']

9.689686793405707

# Step4: Solveing enzyme concentration constraint by COBRApy.

In [101]:
json_model_path="./model/iML1515_irr_enz_constraint.json"
enz_model=get_enzyme_constraint_model(json_model_path)
fba_solution = enz_model.optimize()
fba_solution_df = fba_solution.to_frame()
fba_solution_df.to_csv('./analysis/ECMpy_solution_df_fba.csv')
fba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
#pfba_solution = cobra.flux_analysis.pfba(enz_model)
#pfba_solution_df = pfba_solution.to_frame()
#pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
#pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.8769972144269681

In [3]:
json_model_path="./model/iML1515_irr_enz_constraint.json"
enz_model=get_enzyme_constraint_model(json_model_path)
pfba_solution = cobra.flux_analysis.pfba(enz_model)
pfba_solution_df = pfba_solution.to_frame()
pfba_solution_df.to_csv('./analysis/ECMpy_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']

0.8769972144269681

In [11]:
norm_model=cobra.io.json.load_json_model(json_model_path)
#fba_solution = norm_model.optimize()
#fba_solution_df = fba_solution.to_frame()
#fba_solution_df.to_csv('./analysis/Orimodel_solution_df_fba.csv')
#fba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
pfba_solution = cobra.flux_analysis.pfba(norm_model)
pfba_solution_df = pfba_solution.to_frame()
pfba_solution_df.to_csv('./analysis/Orimodel_solution_df_pfba.csv')
pfba_solution.fluxes['BIOMASS_Ec_iML1515_core_75p37M']


0.8769972144269698